In [ ]:
{
 "cells": [
  {
   "source": [],
   "cell_type": "markdown",
   "metadata": {}
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Understanding the Kugelman et al 2019 Code\n",
    "## Data for this Jupyter notebook\n",
    "These files need to be in the same directory as this jupyter notebook:\n",
    "\n",
    "1. LI = Lab images (Aconitums*.*)\n",
    "2. KEX = (Kugelman et al 2009) example_data.hdf5\n",
    "\n",
    "## Semantic Model\n",
    "The code in https://github.com/jakugel/oct-choroid-seg will be analyzed and modified to read in area segmentations instead of boundaries, and to decide whether to use hdf5 format for the images or read the images directly from a directory.\n",
    "\n",
    "The boundaries are the ILM, RPE and CSI. For the semantic segmentation these 4 regions (areas) are needed:\n",
    "\n",
    "1. Vitreous (top of the image to ILM)\n",
    "2. Retinal (ILM to RPE)\n",
    "3. Choroid (RPE to CHR)\n",
    "4. Sclera (CHR to bottom of the image)\n",
    "\n",
    "## Loading Area Labels\n",
    "The semantic model requires image area segmentations, but the functions `load_training_data` and `load_validation_data` in `train_script_semantic_general.py` are written for segmentation files that contain boundaries. \n",
    "\n",
    "These boundaries are then converted to areas or “masks” in function `create_all_area_masks(images, segs)`.\n",
    "\n",
    "If the input training labels are already area labels, the function `create_all_area_masks` does not need to convert boundaries to areas. It should be enough to set `mask = segs`.\n",
    "\n",
    "## Dependencies\n",
    "See Readme
   ]
  },
  {
   "source": [
    "# Understanding the Images\n",
    "## Directory structure and HDF5 structure\n",
    "\n",
    "LI = Lab images  \n",
    "KEX = (Kugelman et al 2009) example_data.hdf5\n",
    "\n",
    "The KEX file can be read with `hdf52images.py` and has this structure: \n",
    "```\n",
    "test_images\n",
    "Dims: (3, 1536, 496, 1)\n",
    "test_segs\n",
    "Dims: (3, 3, 1536)\n",
    "train_images\n",
    "Dims: (3, 1536, 496, 1)\n",
    "train_segs\n",
    "Dims: (3, 3, 1536)\n",
    "val_images\n",
    "Dims: (3, 1536, 496, 1)\n",
    "val_segs\n",
    "Dims: (3, 3, 1536)\n",
    "```\n",
    "\n",
    "The 4 dimensions (called 'shape' in hdf5 format) of the images are :\n",
    "```\n",
    "num_images = images.shape[0]\n",
    "image_width = images.shape[1]\n",
    "image_height = images.shape[2]\n",
    "num_channels = images.shape[3]\n",
    "```\n",
    "\n",
    "The remlmaterials directory uses the same structure, and the LI images are read from the directory not an hdf5 file.\n",
    "\n",
    "The format is:\n",
    "```\n",
    "val_images\n",
    "3 tif files: (496, 1536)\n",
    "val_segs\n",
    "3 png files: (496, 1536)\n",
    "```\n",
    "... same for test and train datasets.\n",
    "\n",
    "## Differences with example_data.hdf5\n",
    "\n",
    "* KEX images need to be rotated -90 or + 270 degrees to be in the same angle as LI\n",
    "* LI use RGB colors and need to be converted to greyscale\n",
    "* LI image format is hdf5, KEX is TIF for the images and png for the segmentations/labels \n",
    "* KEX uses boundaries as labels, RGI uses areas\n",
    "* LI has scanning instrument label on the images\n",
    "* KEX accounts for RGB or greyscale with the 4th dimension of \"channels\"\n",
    "\n",
    "## Code Incompatibilities\n",
    "The (Kugelman et al 2009) code makes extensive use of the hdf5 format. Some functions, such as `ImageDatabase` in `image_database.py` expect hdf5 datasets as arguments. \n",
    "\n",
    "The code below converts a numpy array to an hdf5 dataset, but requires the creation of an hdf5 file.\n",
    "\n"
   ],
   "cell_type": "markdown",
   "metadata": {}
  },
  {
   "cell_type": "code",
   "execution_count": 57,
   "metadata": {},
   "outputs": [
    {
     "output_type": "stream",
     "name": "stdout",
     "text": [
      "\nLI\n496\n1536\n3\n\nKEX\n3\n1536\n496\n1\n\nLI Reformatted\n3\n1536\n496\n1\n"
     ]
    }
   ],
   "source": [
    "import numpy as np\n",
    "import h5py\n",
    "from PIL import Image, ImageOps\n",
    "\n",
    "# load image as is\n",
    "image = np.array(Image.open('Aconitums004.tif'))\n",
    "h5f = h5py.File('data.h5', 'w')\n",
    "try:\n",
    "    h5imgarray=h5f.create_dataset('train_images', data=image)\n",
    "    print(\"\\nLI\")\n",
    "    # The code below outputs the same dimenions as print(\"Dims: \" + str(dset.shape))\n",
    "    # But clarifies the hdf5 dataset structure\n",
    "    for i in range(h5imgarray.ndim):\n",
    "        print(h5imgarray.shape[i])\n",
    "finally:\n",
    "    h5f.close()\n",
    "\n",
    "#load KEX images\n",
    "h5f = h5py.File('example_data.hdf5', 'r')\n",
    "try:\n",
    "    h5=h5f['train_images'][:]\n",
    "    print(\"\\nKEX\")\n",
    "    for i in range(h5.ndim):\n",
    "        print(h5.shape[i])\n",
    "finally:    \n",
    "    h5f.close()\n",
    "\n",
    "#Modify LI to the same format as KEX\n",
    "train_images = []\n",
    "# Convert to greyscale and rotat\n",
    "train_images.append(np.rot90(np.array(ImageOps.grayscale(Image.open('Aconitums002.tif'))),3))\n",
    "train_images.append(np.rot90(np.array(ImageOps.grayscale(Image.open('Aconitums003.tif'))),3))\n",
    "train_images.append(np.rot90(np.array(ImageOps.grayscale(Image.open('Aconitums004.tif'))),3))\n",
    "# Add the 4th axis with dimension 1 \n",
    "train_images=np.asarray(train_images)\n",
    "train_images=train_images.reshape(3, 1536,496,1)\n",
    "h5f = h5py.File('data.h5', 'w')\n",
    "try:\n",
    "    h5imgarray=h5f.create_dataset('train_images', data=train_images)\n",
    "    print(\"\\nLI Reformatted\")\n",
    "    for i in range(h5imgarray.ndim):\n",
    "        print(h5imgarray.shape[i])\n",
    "\n",
    "finally:\n",
    "    h5f.close()\n"
   ]
  },
  {
   "source": [
    "# BioTeam Code Modifications\n",
    "The BioTeam code can be found in the [BioTeam github](https://github.com/bioteam/ML-Image-Segmentation)\n",
    "## Dependencies\n",
    "environment.yml from [BioTeam's github](https://github.com/bioteam/ML-Image-Segmentation/tree/main/oct-choroid-seg)\n",
    "\n",
    "## Processing images\n",
    "`create_all_area_masks(images, segs)` calls `create_area_mask(image, segs)` for all training or evaluation image and label pairs. In our dataset, images are TIF files, and segs are PNG files.\n",
    "\n",
    "The `images, segs` parameters of `create_all_area_masks` are HDF5 datasets, not regular arrays. The return value is an np.array. \n",
    "\n",
    "We modify `create_area_mask(image, segs)` to set `mask = segs` **if the image and segs dimensions and dtype are the same and dtype is uint8**.\n",
    "\n",
    "<div class=\"alert-danger\">\n",
    "Warning: Matplolib reads png files with function png.read_png_float by default which makes it seem integer png files are of type float32. Use PIL instead. \n",
    "</div>\n",
    "\n",
    "<br>The code below uses PIL to inspect the images:"
   ],
   "cell_type": "markdown",
   "metadata": {}
  },
  {
   "cell_type": "code",
   "execution_count": 59,
   "metadata": {},
   "outputs": [
    {
     "output_type": "stream",
     "name": "stdout",
     "text": [
      "uint8\n(496, 1536, 3)\n"
     ]
    },
    {
     "output_type": "display_data",
     "data": {
      "text/plain": "<Figure size 432x288 with 1 Axes>",
      "image/svg+xml": "<?xml version=\"1.0\" encoding=\"utf-8\" standalone=\"no\"?>\n<!DOCTYPE svg PUBLIC \"-//W3C//DTD SVG 1.1//EN\"\n  \"http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd\">\n<svg height=\"142.880859pt\" version=\"1.1\" viewBox=\"0 0 375.2875 142.880859\" width=\"375.2875pt\" xmlns=\"http://www.w3.org/2000/svg\" xmlns:xlink=\"http://www.w3.org/1999/xlink\">\n <metadata>\n  <rdf:RDF xmlns:cc=\"http://creativecommons.org/ns#\" xmlns:dc=\"http://purl.org/dc/elements/1.1/\" xmlns:rdf=\"http://www.w3.org/1999/02/22-rdf-syntax-ns#\">\n   <cc:Work>\n    <dc:type rdf:resource=\"http://purl.org/dc/dcmitype/StillImage\"/>\n    <dc:date>2021-05-12T13:13:56.916628</dc:date>\n    <dc:format>image/svg+xml</dc:format>\n    <dc:creator>\n     <cc:Agent>\n      <dc:title>Matplotlib v3.4.1, https://matplotlib.org/</dc:title>\n     </cc:Agent>\n    </dc:creator>\n   </cc:Work>\n  </rdf:RDF>\n </metadata>\n <defs>\n  <style type=\"text/css\">*{stroke-linecap:butt;stroke-linejoin:round;}</style>\n </defs>\n <g id=\"figure_1\">\n  <g id=\"patch_1\">\n   <path d=\"M 0 142.880859 \nL 375.2875 142.880859 \nL 375.2875 -0 \nL 0 -0 \nz\n\" style=\"fill:none;\"/>\n  </g>\n  <g id=\"axes_1\">\n   <g id=\"patch_2\">\n    <path d=\"M 33.2875 119.002734 \nL 368.0875 119.002734 \nL 368.0875 10.890234 \nL 33.2875 10.890234 \nz\n\" style=\"fill:#ffffff;\"/>\n   </g>\n   <g clip-path=\"url(#p08bdb8ada8)\">\n    <image height=\"109\" id=\"image0dca8ab5c8\" transform=\"scale(1 -1)translate(0 -109)\" width=\"335\" x=\"33.2875\" xlink:href=\"data:image/png;base64,\...\" y=\"-10.002734\"/>\n   </g>\n   <g id=\"matplotlib.axis_1\">\n    <g id=\"xtick_1\">\n     <g id=\"line2d_1\">\n      <defs>\n       <path d=\"M 0 0 \nL 0 3.5 \n\" id=\"m6ba12c76fa\" style=\"stroke:#000000;stroke-width:0.8;\"/>\n      </defs>\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"33.396484\" xlink:href=\"#m6ba12c76fa\" y=\"119.002734\"/>\n      </g>\n     </g>\n     <g id=\"text_1\">\n      <!-- 0 -->\n      <g transform=\"translate(30.215234 133.601172)scale(0.1 -0.1)\">\n       <defs>\n        <path d=\"M 2034 4250 \nQ 1547 4250 1301 3770 \nQ 1056 3291 1056 2328 \nQ 1056 1369 1301 889 \nQ 1547 409 2034 409 \nQ 2525 409 2770 889 \nQ 3016 1369 3016 2328 \nQ 3016 3291 2770 3770 \nQ 2525 4250 2034 4250 \nz\nM 2034 4750 \nQ 2819 4750 3233 4129 \nQ 3647 3509 3647 2328 \nQ 3647 1150 3233 529 \nQ 2819 -91 2034 -91 \nQ 1250 -91 836 529 \nQ 422 1150 422 2328 \nQ 422 3509 836 4129 \nQ 1250 4750 2034 4750 \nz\n\" id=\"DejaVuSans-30\" transform=\"scale(0.015625)\"/>\n       </defs>\n       <use xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"xtick_2\">\n     <g id=\"line2d_2\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"76.990234\" xlink:href=\"#m6ba12c76fa\" y=\"119.002734\"/>\n      </g>\n     </g>\n     <g id=\"text_2\">\n      <!-- 200 -->\n      <g transform=\"translate(67.446484 133.601172)scale(0.1 -0.1)\">\n       <defs>\n        <path d=\"M 1228 531 \nL 3431 531 \nL 3431 0 \nL 469 0 \nL 469 531 \nQ 828 903 1448 1529 \nQ 2069 2156 2228 2338 \nQ 2531 2678 2651 2914 \nQ 2772 3150 2772 3378 \nQ 2772 3750 2511 3984 \nQ 2250 4219 1831 4219 \nQ 1534 4219 1204 4116 \nQ 875 4013 500 3803 \nL 500 4441 \nQ 881 4594 1212 4672 \nQ 1544 4750 1819 4750 \nQ 2544 4750 2975 4387 \nQ 3406 4025 3406 3419 \nQ 3406 3131 3298 2873 \nQ 3191 2616 2906 2266 \nQ 2828 2175 2409 1742 \nQ 1991 1309 1228 531 \nz\n\" id=\"DejaVuSans-32\" transform=\"scale(0.015625)\"/>\n       </defs>\n       <use xlink:href=\"#DejaVuSans-32\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"xtick_3\">\n     <g id=\"line2d_3\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"120.583984\" xlink:href=\"#m6ba12c76fa\" y=\"119.002734\"/>\n      </g>\n     </g>\n     <g id=\"text_3\">\n      <!-- 400 -->\n      <g transform=\"translate(111.040234 133.601172)scale(0.1 -0.1)\">\n       <defs>\n        <path d=\"M 2419 4116 \nL 825 1625 \nL 2419 1625 \nL 2419 4116 \nz\nM 2253 4666 \nL 3047 4666 \nL 3047 1625 \nL 3713 1625 \nL 3713 1100 \nL 3047 1100 \nL 3047 0 \nL 2419 0 \nL 2419 1100 \nL 313 1100 \nL 313 1709 \nL 2253 4666 \nz\n\" id=\"DejaVuSans-34\" transform=\"scale(0.015625)\"/>\n       </defs>\n       <use xlink:href=\"#DejaVuSans-34\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"xtick_4\">\n     <g id=\"line2d_4\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"164.177734\" xlink:href=\"#m6ba12c76fa\" y=\"119.002734\"/>\n      </g>\n     </g>\n     <g id=\"text_4\">\n      <!-- 600 -->\n      <g transform=\"translate(154.633984 133.601172)scale(0.1 -0.1)\">\n       <defs>\n        <path d=\"M 2113 2584 \nQ 1688 2584 1439 2293 \nQ 1191 2003 1191 1497 \nQ 1191 994 1439 701 \nQ 1688 409 2113 409 \nQ 2538 409 2786 701 \nQ 3034 994 3034 1497 \nQ 3034 2003 2786 2293 \nQ 2538 2584 2113 2584 \nz\nM 3366 4563 \nL 3366 3988 \nQ 3128 4100 2886 4159 \nQ 2644 4219 2406 4219 \nQ 1781 4219 1451 3797 \nQ 1122 3375 1075 2522 \nQ 1259 2794 1537 2939 \nQ 1816 3084 2150 3084 \nQ 2853 3084 3261 2657 \nQ 3669 2231 3669 1497 \nQ 3669 778 3244 343 \nQ 2819 -91 2113 -91 \nQ 1303 -91 875 529 \nQ 447 1150 447 2328 \nQ 447 3434 972 4092 \nQ 1497 4750 2381 4750 \nQ 2619 4750 2861 4703 \nQ 3103 4656 3366 4563 \nz\n\" id=\"DejaVuSans-36\" transform=\"scale(0.015625)\"/>\n       </defs>\n       <use xlink:href=\"#DejaVuSans-36\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"xtick_5\">\n     <g id=\"line2d_5\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"207.771484\" xlink:href=\"#m6ba12c76fa\" y=\"119.002734\"/>\n      </g>\n     </g>\n     <g id=\"text_5\">\n      <!-- 800 -->\n      <g transform=\"translate(198.227734 133.601172)scale(0.1 -0.1)\">\n       <defs>\n        <path d=\"M 2034 2216 \nQ 1584 2216 1326 1975 \nQ 1069 1734 1069 1313 \nQ 1069 891 1326 650 \nQ 1584 409 2034 409 \nQ 2484 409 2743 651 \nQ 3003 894 3003 1313 \nQ 3003 1734 2745 1975 \nQ 2488 2216 2034 2216 \nz\nM 1403 2484 \nQ 997 2584 770 2862 \nQ 544 3141 544 3541 \nQ 544 4100 942 4425 \nQ 1341 4750 2034 4750 \nQ 2731 4750 3128 4425 \nQ 3525 4100 3525 3541 \nQ 3525 3141 3298 2862 \nQ 3072 2584 2669 2484 \nQ 3125 2378 3379 2068 \nQ 3634 1759 3634 1313 \nQ 3634 634 3220 271 \nQ 2806 -91 2034 -91 \nQ 1263 -91 848 271 \nQ 434 634 434 1313 \nQ 434 1759 690 2068 \nQ 947 2378 1403 2484 \nz\nM 1172 3481 \nQ 1172 3119 1398 2916 \nQ 1625 2713 2034 2713 \nQ 2441 2713 2670 2916 \nQ 2900 3119 2900 3481 \nQ 2900 3844 2670 4047 \nQ 2441 4250 2034 4250 \nQ 1625 4250 1398 4047 \nQ 1172 3844 1172 3481 \nz\n\" id=\"DejaVuSans-38\" transform=\"scale(0.015625)\"/>\n       </defs>\n       <use xlink:href=\"#DejaVuSans-38\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"xtick_6\">\n     <g id=\"line2d_6\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"251.365234\" xlink:href=\"#m6ba12c76fa\" y=\"119.002734\"/>\n      </g>\n     </g>\n     <g id=\"text_6\">\n      <!-- 1000 -->\n      <g transform=\"translate(238.640234 133.601172)scale(0.1 -0.1)\">\n       <defs>\n        <path d=\"M 794 531 \nL 1825 531 \nL 1825 4091 \nL 703 3866 \nL 703 4441 \nL 1819 4666 \nL 2450 4666 \nL 2450 531 \nL 3481 531 \nL 3481 0 \nL 794 0 \nL 794 531 \nz\n\" id=\"DejaVuSans-31\" transform=\"scale(0.015625)\"/>\n       </defs>\n       <use xlink:href=\"#DejaVuSans-31\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"190.869141\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"xtick_7\">\n     <g id=\"line2d_7\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"294.958984\" xlink:href=\"#m6ba12c76fa\" y=\"119.002734\"/>\n      </g>\n     </g>\n     <g id=\"text_7\">\n      <!-- 1200 -->\n      <g transform=\"translate(282.233984 133.601172)scale(0.1 -0.1)\">\n       <use xlink:href=\"#DejaVuSans-31\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-32\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"190.869141\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"xtick_8\">\n     <g id=\"line2d_8\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"338.552734\" xlink:href=\"#m6ba12c76fa\" y=\"119.002734\"/>\n      </g>\n     </g>\n     <g id=\"text_8\">\n      <!-- 1400 -->\n      <g transform=\"translate(325.827734 133.601172)scale(0.1 -0.1)\">\n       <use xlink:href=\"#DejaVuSans-31\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-34\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"190.869141\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n   </g>\n   <g id=\"matplotlib.axis_2\">\n    <g id=\"ytick_1\">\n     <g id=\"line2d_9\">\n      <defs>\n       <path d=\"M 0 0 \nL -3.5 0 \n\" id=\"m69d1ed90e3\" style=\"stroke:#000000;stroke-width:0.8;\"/>\n      </defs>\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"33.2875\" xlink:href=\"#m69d1ed90e3\" y=\"10.999219\"/>\n      </g>\n     </g>\n     <g id=\"text_9\">\n      <!-- 0 -->\n      <g transform=\"translate(19.925 14.798437)scale(0.1 -0.1)\">\n       <use xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"ytick_2\">\n     <g id=\"line2d_10\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"33.2875\" xlink:href=\"#m69d1ed90e3\" y=\"32.796094\"/>\n      </g>\n     </g>\n     <g id=\"text_10\">\n      <!-- 100 -->\n      <g transform=\"translate(7.2 36.595312)scale(0.1 -0.1)\">\n       <use xlink:href=\"#DejaVuSans-31\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"ytick_3\">\n     <g id=\"line2d_11\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"33.2875\" xlink:href=\"#m69d1ed90e3\" y=\"54.592969\"/>\n      </g>\n     </g>\n     <g id=\"text_11\">\n      <!-- 200 -->\n      <g transform=\"translate(7.2 58.392187)scale(0.1 -0.1)\">\n       <use xlink:href=\"#DejaVuSans-32\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"ytick_4\">\n     <g id=\"line2d_12\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"33.2875\" xlink:href=\"#m69d1ed90e3\" y=\"76.389844\"/>\n      </g>\n     </g>\n     <g id=\"text_12\">\n      <!-- 300 -->\n      <g transform=\"translate(7.2 80.189062)scale(0.1 -0.1)\">\n       <defs>\n        <path d=\"M 2597 2516 \nQ 3050 2419 3304 2112 \nQ 3559 1806 3559 1356 \nQ 3559 666 3084 287 \nQ 2609 -91 1734 -91 \nQ 1441 -91 1130 -33 \nQ 819 25 488 141 \nL 488 750 \nQ 750 597 1062 519 \nQ 1375 441 1716 441 \nQ 2309 441 2620 675 \nQ 2931 909 2931 1356 \nQ 2931 1769 2642 2001 \nQ 2353 2234 1838 2234 \nL 1294 2234 \nL 1294 2753 \nL 1863 2753 \nQ 2328 2753 2575 2939 \nQ 2822 3125 2822 3475 \nQ 2822 3834 2567 4026 \nQ 2313 4219 1838 4219 \nQ 1578 4219 1281 4162 \nQ 984 4106 628 3988 \nL 628 4550 \nQ 988 4650 1302 4700 \nQ 1616 4750 1894 4750 \nQ 2613 4750 3031 4423 \nQ 3450 4097 3450 3541 \nQ 3450 3153 3228 2886 \nQ 3006 2619 2597 2516 \nz\n\" id=\"DejaVuSans-33\" transform=\"scale(0.015625)\"/>\n       </defs>\n       <use xlink:href=\"#DejaVuSans-33\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"ytick_5\">\n     <g id=\"line2d_13\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"33.2875\" xlink:href=\"#m69d1ed90e3\" y=\"98.186719\"/>\n      </g>\n     </g>\n     <g id=\"text_13\">\n      <!-- 400 -->\n      <g transform=\"translate(7.2 101.985937)scale(0.1 -0.1)\">\n       <use xlink:href=\"#DejaVuSans-34\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n   </g>\n   <g id=\"patch_3\">\n    <path d=\"M 33.2875 119.002734 \nL 33.2875 10.890234 \n\" style=\"fill:none;stroke:#000000;stroke-linecap:square;stroke-linejoin:miter;stroke-width:0.8;\"/>\n   </g>\n   <g id=\"patch_4\">\n    <path d=\"M 368.0875 119.002734 \nL 368.0875 10.890234 \n\" style=\"fill:none;stroke:#000000;stroke-linecap:square;stroke-linejoin:miter;stroke-width:0.8;\"/>\n   </g>\n   <g id=\"patch_5\">\n    <path d=\"M 33.2875 119.002734 \nL 368.0875 119.002734 \n\" style=\"fill:none;stroke:#000000;stroke-linecap:square;stroke-linejoin:miter;stroke-width:0.8;\"/>\n   </g>\n   <g id=\"patch_6\">\n    <path d=\"M 33.2875 10.890234 \nL 368.0875 10.890234 \n\" style=\"fill:none;stroke:#000000;stroke-linecap:square;stroke-linejoin:miter;stroke-width:0.8;\"/>\n   </g>\n  </g>\n </g>\n <defs>\n  <clipPath id=\"p08bdb8ada8\">\n   <rect height=\"108.1125\" width=\"334.8\" x=\"33.2875\" y=\"10.890234\"/>\n  </clipPath>\n </defs>\n</svg>\n",
      "image/png": "...\n"
     },
     "metadata": {
      "needs_background": "light"
     }
    },
    {
     "output_type": "stream",
     "name": "stdout",
     "text": [
      "uint8\n(496, 1536, 3)\n"
     ]
    },
    {
     "output_type": "display_data",
     "data": {
      "text/plain": "<Figure size 432x288 with 1 Axes>",
      "image/svg+xml": "<?xml version=\"1.0\" encoding=\"utf-8\" standalone=\"no\"?>\n<!DOCTYPE svg PUBLIC \"-//W3C//DTD SVG 1.1//EN\"\n  \"http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd\">\n<svg height=\"142.880859pt\" version=\"1.1\" viewBox=\"0 0 375.2875 142.880859\" width=\"375.2875pt\" xmlns=\"http://www.w3.org/2000/svg\" xmlns:xlink=\"http://www.w3.org/1999/xlink\">\n <metadata>\n  <rdf:RDF xmlns:cc=\"http://creativecommons.org/ns#\" xmlns:dc=\"http://purl.org/dc/elements/1.1/\" xmlns:rdf=\"http://www.w3.org/1999/02/22-rdf-syntax-ns#\">\n   <cc:Work>\n    <dc:type rdf:resource=\"http://purl.org/dc/dcmitype/StillImage\"/>\n    <dc:date>2021-05-12T13:13:57.399076</dc:date>\n    <dc:format>image/svg+xml</dc:format>\n    <dc:creator>\n     <cc:Agent>\n      <dc:title>Matplotlib v3.4.1, https://matplotlib.org/</dc:title>\n     </cc:Agent>\n    </dc:creator>\n   </cc:Work>\n  </rdf:RDF>\n </metadata>\n <defs>\n  <style type=\"text/css\">*{stroke-linecap:butt;stroke-linejoin:round;}</style>\n </defs>\n <g id=\"figure_1\">\n  <g id=\"patch_1\">\n   <path d=\"M 0 142.880859 \nL 375.2875 142.880859 \nL 375.2875 -0 \nL 0 -0 \nz\n\" style=\"fill:none;\"/>\n  </g>\n  <g id=\"axes_1\">\n   <g id=\"patch_2\">\n    <path d=\"M 33.2875 119.002734 \nL 368.0875 119.002734 \nL 368.0875 10.890234 \nL 33.2875 10.890234 \nz\n\" style=\"fill:#ffffff;\"/>\n   </g>\n   <g clip-path=\"url(#pc3fd492f59)\">\n    <image height=\"109\" id=\"image75a324306e\" transform=\"scale(1 -1)translate(0 -109)\" width=\"335\" x=\"33.2875\" xlink:href=\"data:image/png;base64,\niVBORw0KGgoAAAANSUhEUgAAAU8AAABtCAYAAAAh3uN7AAAVXUlEQVR4nO3dWYyk11338e951tqrq3qf7pnp2ewkHmMnYxwnxokJSljCBSIIEAFFLAIkhLhAcAHcICGBogSJC3ijV/Ci971BvCQICBIoUgxEwk5wHOIZZ+xZ3d0zvVbXvj/b4aK6e3rssaen3D3d1fX/3JWep6tPdT/Pr845z1kUoBFCCPFAjIMugBBCDCIJTyGE6IOEpxBC9EHCUwgh+iDhKYQQfZDwFEKIPkh4CiFEHyQ8hRCiDxKeQgjRBwlPIYTog4SnEEL0QcJTCCH6IOEphBB9kPAUQog+SHgKIUQfJDyFEKIPEp5CCNEHCU8hhOiDhKcQQvRBwlMIIfog4SmEEH2Q8BRCiD5IeAohRB8kPIUQog8SnkII0QcJTyGE6IOEpxBC9EHCUwgh+iDhKYQQfZDwFEKIPlgHXQCxt5QysA2XhJMlGxsnEx8nGxsHoN4t0vSq2+cGoUe1s37nh7Wm5deIorCPXwxxO4NpWJsvFdn4BKZhA2AbDvnkDArV1+fywg7l1gqhDqi01tA63C5roH20jvp6XyH6JeF5SCll3DNoTMPGMWPbr10rSTqWJ5+YYTQ5Q8LJkHHHiDtpTMNBoVCq9z5a67e8myaMgh2vNB2/QaT7CE8gZqcw1Z1Lqhekd3+GrbI8qJ1lDyMfjabt14l0RKNbxAs6NL0yjW6FbtCk2i4Qap96p4hG0/WbaHTfn02It1LAW+8o8ZBYhkPCyTCaPM546jgxKwX0gjOXmN6uxe3kmDHidnr7tWlYWIYDO0Jy2Gmt0Wi0jvDDDlpHNLplIh1Saa/hh13afoNap4Afdam219E6oulV0DoiiHwJWXFfEp4PiaFMYnaKbGyc0eQs09lz5BPTJJ0crhVHwu/hulOT1YQ6ROuIjt9Ao2l2K3TDFi2vRqNbBCDSEaXmEuGOUG15VbpBC601Hb9OqIN7/CZxVEl47gOFwjZjpNwc46kTjKVOMpacJZeYxrUSGMqUoBwwvbC9+1bZqqFqHVHtFFir3WSx/D2Kzdu0/Jr0wx5xEp4PyFQWKEXcTqEwScfy2KZL2h0l4WSI2WmysXHSsVGSThbLcIH++/rE4Oh1F/RqsIXGIrcrb7Bau0GptYwfdg66eGKPDV14KgwcK4ZCYSiTlJuHzWCzDIdcfHL7NUB8MwwBDMMiF59CKYOkk8VQJqbhYKjeiC8JSLGT1ppQB9Q7G6zXF1gsv8Z6fZ6GVyaM/IMunniPhio8z4w9xfnpj5OJjW02nQ1cK7F9vPdk2pAQFPtC6wgv7FBprXK78gYrteusNxbobva1isEyVEOV0m6emZFHD7oYYkhtfVlPZk4zkT6FJqLpVSk2bnO78jrL1avUOgU8aeIPhKEKTyEOC6VUr8/czZN285zMP04Q+TS6RZarV1muXqPQWKDeLUkT/5CS8BTiEFBKYZsOucQ0I/EpPjD1MbywQ7W9zlr9TVZq1yjUF6S/9BCR8BTikNnqc3etOBPpk0ykT3J++u4wXa5epdBYpNkty/jSAyLhKcQA2OovnUjPMZ46yfnpj/cePrXXWKtt1kwlTB8qCU8hBkyvZqp6D5/Sp5hIzfH4sefxgjaVzjq3ypdZKF2k2FwiiLyDLu6RJeEpxIDbDlM7yaTdC9MnZz5Jub3CQukS88WLlFrLEqR7TMLzCNE6esfRgor9W7zV1h7GPkxF1EAjutcISM3O32aZLqayZHzuJqUUlukwnjrJWPIET858ilJrhYXSRRZKlyi1lgjkodN7JuF5SO1cgm1r/nSoAzy/iaOg0a3gRB3sqEO1vcqcadHubhALW5w23/5vTRgGZy2rz9U0310mqGLvw80YAItBwFtjua011wJ/O1SbVoZm/Dit+AxGYhbbGQGjt2zfsAfqVpBOpE8ynjrBk7OfotRaZqF0iYXSRUqtFXl636ehmmH05Myn+OjpnzrQMmwtlwaaMOyidYgfeuDXsBU0OxuM6S5B2MborDNjmjS6JU7gESNE+w1OWRZR5JNWmpQyAE1cqbtqlsMUGlr3aqJtDSVMunaGmjtFwT3GRnyWsp2nZSaJMO6aejvMtNYEkUepucxK7RrzpYsUGosyB/8BSHjuhc1aokJjbT7pTIRNTB3iBw0moyZh0KHeXuV9lkW93fu2Hw8bJLRHqENGtEdaGUBERqntZrYpN3vfNBBh0DbjlO08q+4xlmKzrDsT1Kwsvuqtcj/sgap1b5HoanudxfJrvFl8lY3mLQnS+5Dw7IPSEbGoQ94rMuWtkAmqZP0qjvYY8csAJMMGho7QOsTWUe/+3ArZIb9ZD8rWhe4rh5qVZt2ZZDk2w6p7jLKdJ1AWnuEADG0ttRekAZX2Oovl7zFf/O5mkHYPumiHjoTn/WiNQuNGHUb9IjOd2xxvLzLhrZIKGpiE+9KPKB6Ordpp14jRNVyaZhIUlOxRPMOlZfRqrQAVO4dnOPjKpm0m0ECgbHpXSM9R+mLcWhWq2l5jofQa86WLFCVIt0l43oPSEW7UJe8XOda5zYnOIpPdVVJBXcJyyOy8OUJMtFIEyqZtxNDKoGzn8TB4Q9tcM0e4GYaYsSmUYWGbMZLOCLYZwzFdlBrczWq3grTSWmWh/BrXCy9Tbq0SDfGAfAnPzaa0G3XI+6VezXIzLNNhDVNLWIr701rjA+thyOUg4N+6HV71Q5a1SWQlGUudZDp7jsn0KbKxcWxzcEcDaK3xww6FxiJX1r/J7crrNLqlgy7WQzd84XnqMwC4UZecX2Km22uGT3VXJCzFntgaZtbWmqUw5KLv8UKnwyXfZx0D28kzsRmmo8kZRuKTOFbirp1OB8HW52x5VW5VLnNz479Zrl7FC9sHXLKHY6jC87mpZ/jcxOOcaC8w2V0hHdaxdCBhKfZVb3gaNLRmPgj4ju/xqudxLQhY1wrsDInEDFOZc0xmzjASnxy4TQG3tiCptNZYKF3iauG/qLRWjvQ8+6EKz99JZ/nNdFrCUhy4rWZ+JYqoRRFLYciNIGAhgpKZohk7hps8Tjw1RzI+RcxOoRiMXQ56Y0i7rNfnuVZ4mduV16l3ikdutfyhCs8v5fJ8Op64/4lCHJCtWmpbaxpaU4hg2YjTcicJk6eoJU5Sc8Y3n/arQz+cSmtNJ2hyq/w9bmy8wlLlypFp1g9NeJrA/xsd42Nu7KCLIkRftIZQGbTMJCVnlDVnilV3moIzTtnMEJgu+hDXTqMopNop8Gbxu1xb/xbF1tJBF+k9GZrwjCvFv45PcNqyD7oo2zSwaDhsmBZWegx1j7IlvTanKsvvuqhHBET7MAwmMCxatrurcxtOgkosdc9j6W6LbLcBgNKalN9GbT5sMLTGGI5LcM/dGfRvUTWSrNo5btgTLDqTVJwJlJPHMN1D9yBKa81K7Tr/eOmLA723/dAsDBJXitghuoAAivEsv//xX2U+OYrhJlHG2wMw4Xf47W/9DZ+Y//Y9AzRC8Xfv/wT/ceLJPS9fy45RjGd3da5n2nQs557HnNAnFvSWQzN0xFSzhBmFAIy2a4y3erOyrCjiTPk2lu4dy7Xr5Nu13s8RMdqubYeuHQVYm++x5XD9d/ff1ud1dMB4WGU8rHK+/SYeig1tsG5luG7mmHcmKbvTEJsg5mSxjN4X4kEFqlK9bb8H3dCE54hhMPIQBylrIFIKQ+t73tQRir957JPMj58Gpd62ctCWupPgC8/8HMDbAlQDL8xd4K+e+HFaTnxvP8AeahsmbftOd0kxMXLvE/XdNVBLh9sha2hNrlPfXvou366R69R750UhZ8pLfHDtKo8Xbu79BxggSilcYEZFzEQVngzLBN5NijXNjcjgNeLcsPKU4rNkMo+SiU/imLGHHqSulcAynIGePz804RlDYezTBaKBwDBpOAk24lnms1NcHT3B7fQEv/HKV5ipF+4KUA1cmjjDP5999v4d/krR2AxQQ2ueX3gFY/M9roye4M8vfOZQB+cDecvfIlAWgXHnEt0ZwLczk3ed+/W5p/i9F//v0IfnWymlsIEpUzFlwkd1C58WS40FXq/+JxdJspw8izNynlRiFseKP5SZUDE7iS3hORjmLIvd9d69Ow34hkXNTbKeyHEzd4xr+eNcz82ylB6j5qbwdtzwgWHyh9/4SxLBnfnADSfBX1z4SZr27h9eNew4n3/msyg0zy98h0I8y58+/bMU3qkWN2QUmtHNJv5uaXo1ezf0scMAxb1bCUeJUgoHOGVZzJmaH6FD07vE7dVLfJcEt2LHaWceQ6VOYVrJQ9dfepgMTXgmH/AC2GpAdk2HhhNnOTXGjdwMV0ZPcCM3w2pylKYdxzOtd609fmvmMf76iU/z69/5B0wdEQF/+8hzXB6be7BhJkrRcBN8/pnP4iuDFybOcXns1KEfqvKwxAKP6UZx1+drYCU1yu89/+soYLa2ziOlW8zW1pmrrjDWqhAPupg6OrKBqjaXPkwrxfsNeJ9uE/lXqW1c5c1Skpv2FIX0+6klTxPZWbQE6V2GJjwfsd/5KXtvaWKFb1qUY2lupSe4np/lSv4E8yPTlOIZak6SwNjs5H6ACyhSBl959HlOVVb40Rsvcc1N8+VzP4Dus2lUd5P88bOfI1DDuWTaOzF1hBvsfo+eDTvOn3zkF7iemwWluJY/zr+d/BAKTSzwyHabzNQLnC3d4tHSLaYbGxxrbJDutrCj3qyZo/bXV0phAjkFOVp80LuBX7xJuRxn1Z3mRuIMheQ5KnZuaJfs22lowjOpet+aW0HZsRyK8SyLmQmu549zNX+cW5kJNhI5mnaMcA/DybMcvvTBn2C2ts5fn32WemrsPd14vnl4hlsdFqOtKhmvuatzW1HEH/mab+dm7+4HV72Ge9uO0bZjrKZGeWXqUaD3dD/dbTHernCyuspks8Rks8Roq8pYu0qm2yTltXFDDzsMMI5AF8BWE39St5ns3OT7OjfpVr7BijvNa85xXk+cw49NYRgPHiO24ZJwsrT8B+tqOUyGIjwNwDVMXpj+ANemznEtN8tqapSNxAgtK0Z0jyFCe60Uz/AHz/8qFTclTZ99kAg6bxu6dC++1nzBSvLiRz6L4ybv/8ab/yvftCklspQSWa6Mntw+bEQRho6IB13cwCPfqZP2mkw3iox0GszU18l2m0w2S2S7TXKdOk7Y2zNo0K4CBcSiLqfa88y15nm68iKvx+a4nHmCWvIckeHs+to2DAvHGuwHnUMRnhHw+40mox/6Saz8zME0N5SitMsxk+LBnaiu3Tc8Q635OzfL137w13AmTu/Jl1hkGEQY1E2LuptkI5m7c3DH0Csn9ImFPhPNEqcryzxaXOSR0iInqqtkvBZWNFireSkFk4RMdK7zTPcmi+Uxvpl8jNXM4wT2yFBUEIYiPAG6hoG+z8MdMbjiwbuvbq6BFyfO8L+f+mn02MmHc3Pv+B2e5eBZDjU3yfX8cb526mmsKCTTbXKsscHZ8m0eLS5yurLEVKNExmsORKAqpYiheSQocKby7xTqL/Na/Cw3Rp6i6EyilHlk77mhCU8zMYKZkJrfUXW6svyOxzTw3clzfPG5XyaIZw9HrUgpAtPa7gp4bfw00OtbzXabHKsXOF+4ySOlW8xVV5holkl57UM9ldVUMBU1mWy+ykea3+N6fI5L6SdYTZ7FU7tv0g+KoQlP1OFfgUb0R+mIkU5ju5a2M166ps16Ms8XP/xzbMSzh/ca2NG3upEYYSMxwsXJc70tYQKfXKfOufItPrh6lQ+tXmWmvo4b+oeyZqqAJAHf17rG+fZNluxxLmee4Hrq/dTNDCiFQpGLT7FcvXrQxe3b8ISnOLIMranEUvzL6WfYSGTxTJubIzNESvUeDMazlGPpwxuc70Irg47tsmK7rKTH+MbxJ4kFHierqzy9cpkLK29wtrxEpts8dLXS3tCniBP+GseLX+PZyovcSJzjvzMXKLgT2OZeTFs5OEOzqpKVn2H6F/8M4wFm9YgBobf2r1ToAQzIvmmNFYWMtqs8XrjBUytvcGHlCmPtClYYHsrvCg34yuZ67AS/VVrnjfLrB12kvknNUww+NQwTK+9hs990LTXKWjLP1+e+n0y3yVxxngvXXuIzS5fIhrufOPAwKMDRPtn6FVYr6wddnPdkcPdCFULcoXq17mosxasz5/k/H/sl/v4DP7TrZqUGIq3xtaYZRdubu+2XtSikMcBrecIQ1TyVaTF4w5KF6JNh8s9nPsqnr/wHE17rrkMhikIyx63MBB3T4c3cMWp2nCu+T6vToLNyjf/VWCazjz16gx2bPUMTnnZuBvUOi/UKcRStpUb56szj/NLNb24PE9LAN048yRee+Sx1J35nB4Kt41pjtqtc+qc/4lmvsW9luxEEDPq+msPTbH8IUzCFOFQMg6+e+jDrm6u2b413/bOnf4aqmyQyzLcN4VNKETpJ/gGLaB+b7p197hZ4GCRRhDjCNqbO8dXxM2itWU6N8adP/+z9x7uaFt9OjVOM9qdxrbVmPbz/OgSHnYSnEEeYNi3+6dSHuWnH+fxHfp43R47tarxrwY7zstfdtwdHt49AeA5Nn6cQw0gpRfH0BX43P836+O4XzzbHT/Llyy/ww7E4O7dqi7QmBFpa09aapTBgzDA5ZppY7H5TueAINNslPIU44pQTZ23izO7PVwozO8ErQcBLXpdaFFGNNG8EPuUoZD4IKUUhtc1hTUnD4H2WzQ/GYjznuJyyLBLqnVed9+g9MBp0QxOevaFKQojdMJN5NlB8rriBz7tPQ/SiiJe8Li95XWIoTlsWn4jFeM51+YDtkL1HkIZHYGLj0CSKPT530EUQYmDYuWlSH/oxom6L3exbELVqhPUCfrPK616Ly80GX2o0mDUNnnNj/IDrcsFxGTMMalFEeZ8eRj1MQxOeyjCP3JJYQuwX5cTJfeJXdv8DWkMUEnWbRN0mQWWNoLJCuVXlK4UFvtwsM9os81jY4ryhqEqfpxDiKHrgioZSYBiY1ghmcgQ7PwNw52l9FKD9LpcaZS56bdwbL9N58W9hgKdoSngKIfbNdgibNsq0MWIpALSOqL70/wd6TTcZ5ymEEH2Q8BRCiD5IeAohRB+GIzyVwh6dPehSCCGOkOEITxTKjh90IYQQR8iQhKcQQuwtCU8hhOiDhKcQQvRBwlMIIfog4SmEEH0YivBUThwzM3bQxRBCHCHDEZ6GiWG5B10MIcQRMhThKYQQe03CUwgh+iDhKYQQfZDwFEKIPkh4CiFEH4YiPM1UHuXKwiBCiL0zFOGpbBdlyI4jQoi9MxThKYQQe02qY2JgaK0JSktE3eZBF0W8R97GYm+74gH2P9udoSuQPZGyAAAAAElFTkSuQmCC\" y=\"-10.002734\"/>\n   </g>\n   <g id=\"matplotlib.axis_1\">\n    <g id=\"xtick_1\">\n     <g id=\"line2d_1\">\n      <defs>\n       <path d=\"M 0 0 \nL 0 3.5 \n\" id=\"m982745fefd\" style=\"stroke:#000000;stroke-width:0.8;\"/>\n      </defs>\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"33.396484\" xlink:href=\"#m982745fefd\" y=\"119.002734\"/>\n      </g>\n     </g>\n     <g id=\"text_1\">\n      <!-- 0 -->\n      <g transform=\"translate(30.215234 133.601172)scale(0.1 -0.1)\">\n       <defs>\n        <path d=\"M 2034 4250 \nQ 1547 4250 1301 3770 \nQ 1056 3291 1056 2328 \nQ 1056 1369 1301 889 \nQ 1547 409 2034 409 \nQ 2525 409 2770 889 \nQ 3016 1369 3016 2328 \nQ 3016 3291 2770 3770 \nQ 2525 4250 2034 4250 \nz\nM 2034 4750 \nQ 2819 4750 3233 4129 \nQ 3647 3509 3647 2328 \nQ 3647 1150 3233 529 \nQ 2819 -91 2034 -91 \nQ 1250 -91 836 529 \nQ 422 1150 422 2328 \nQ 422 3509 836 4129 \nQ 1250 4750 2034 4750 \nz\n\" id=\"DejaVuSans-30\" transform=\"scale(0.015625)\"/>\n       </defs>\n       <use xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"xtick_2\">\n     <g id=\"line2d_2\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"76.990234\" xlink:href=\"#m982745fefd\" y=\"119.002734\"/>\n      </g>\n     </g>\n     <g id=\"text_2\">\n      <!-- 200 -->\n      <g transform=\"translate(67.446484 133.601172)scale(0.1 -0.1)\">\n       <defs>\n        <path d=\"M 1228 531 \nL 3431 531 \nL 3431 0 \nL 469 0 \nL 469 531 \nQ 828 903 1448 1529 \nQ 2069 2156 2228 2338 \nQ 2531 2678 2651 2914 \nQ 2772 3150 2772 3378 \nQ 2772 3750 2511 3984 \nQ 2250 4219 1831 4219 \nQ 1534 4219 1204 4116 \nQ 875 4013 500 3803 \nL 500 4441 \nQ 881 4594 1212 4672 \nQ 1544 4750 1819 4750 \nQ 2544 4750 2975 4387 \nQ 3406 4025 3406 3419 \nQ 3406 3131 3298 2873 \nQ 3191 2616 2906 2266 \nQ 2828 2175 2409 1742 \nQ 1991 1309 1228 531 \nz\n\" id=\"DejaVuSans-32\" transform=\"scale(0.015625)\"/>\n       </defs>\n       <use xlink:href=\"#DejaVuSans-32\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"xtick_3\">\n     <g id=\"line2d_3\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"120.583984\" xlink:href=\"#m982745fefd\" y=\"119.002734\"/>\n      </g>\n     </g>\n     <g id=\"text_3\">\n      <!-- 400 -->\n      <g transform=\"translate(111.040234 133.601172)scale(0.1 -0.1)\">\n       <defs>\n        <path d=\"M 2419 4116 \nL 825 1625 \nL 2419 1625 \nL 2419 4116 \nz\nM 2253 4666 \nL 3047 4666 \nL 3047 1625 \nL 3713 1625 \nL 3713 1100 \nL 3047 1100 \nL 3047 0 \nL 2419 0 \nL 2419 1100 \nL 313 1100 \nL 313 1709 \nL 2253 4666 \nz\n\" id=\"DejaVuSans-34\" transform=\"scale(0.015625)\"/>\n       </defs>\n       <use xlink:href=\"#DejaVuSans-34\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"xtick_4\">\n     <g id=\"line2d_4\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"164.177734\" xlink:href=\"#m982745fefd\" y=\"119.002734\"/>\n      </g>\n     </g>\n     <g id=\"text_4\">\n      <!-- 600 -->\n      <g transform=\"translate(154.633984 133.601172)scale(0.1 -0.1)\">\n       <defs>\n        <path d=\"M 2113 2584 \nQ 1688 2584 1439 2293 \nQ 1191 2003 1191 1497 \nQ 1191 994 1439 701 \nQ 1688 409 2113 409 \nQ 2538 409 2786 701 \nQ 3034 994 3034 1497 \nQ 3034 2003 2786 2293 \nQ 2538 2584 2113 2584 \nz\nM 3366 4563 \nL 3366 3988 \nQ 3128 4100 2886 4159 \nQ 2644 4219 2406 4219 \nQ 1781 4219 1451 3797 \nQ 1122 3375 1075 2522 \nQ 1259 2794 1537 2939 \nQ 1816 3084 2150 3084 \nQ 2853 3084 3261 2657 \nQ 3669 2231 3669 1497 \nQ 3669 778 3244 343 \nQ 2819 -91 2113 -91 \nQ 1303 -91 875 529 \nQ 447 1150 447 2328 \nQ 447 3434 972 4092 \nQ 1497 4750 2381 4750 \nQ 2619 4750 2861 4703 \nQ 3103 4656 3366 4563 \nz\n\" id=\"DejaVuSans-36\" transform=\"scale(0.015625)\"/>\n       </defs>\n       <use xlink:href=\"#DejaVuSans-36\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"xtick_5\">\n     <g id=\"line2d_5\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"207.771484\" xlink:href=\"#m982745fefd\" y=\"119.002734\"/>\n      </g>\n     </g>\n     <g id=\"text_5\">\n      <!-- 800 -->\n      <g transform=\"translate(198.227734 133.601172)scale(0.1 -0.1)\">\n       <defs>\n        <path d=\"M 2034 2216 \nQ 1584 2216 1326 1975 \nQ 1069 1734 1069 1313 \nQ 1069 891 1326 650 \nQ 1584 409 2034 409 \nQ 2484 409 2743 651 \nQ 3003 894 3003 1313 \nQ 3003 1734 2745 1975 \nQ 2488 2216 2034 2216 \nz\nM 1403 2484 \nQ 997 2584 770 2862 \nQ 544 3141 544 3541 \nQ 544 4100 942 4425 \nQ 1341 4750 2034 4750 \nQ 2731 4750 3128 4425 \nQ 3525 4100 3525 3541 \nQ 3525 3141 3298 2862 \nQ 3072 2584 2669 2484 \nQ 3125 2378 3379 2068 \nQ 3634 1759 3634 1313 \nQ 3634 634 3220 271 \nQ 2806 -91 2034 -91 \nQ 1263 -91 848 271 \nQ 434 634 434 1313 \nQ 434 1759 690 2068 \nQ 947 2378 1403 2484 \nz\nM 1172 3481 \nQ 1172 3119 1398 2916 \nQ 1625 2713 2034 2713 \nQ 2441 2713 2670 2916 \nQ 2900 3119 2900 3481 \nQ 2900 3844 2670 4047 \nQ 2441 4250 2034 4250 \nQ 1625 4250 1398 4047 \nQ 1172 3844 1172 3481 \nz\n\" id=\"DejaVuSans-38\" transform=\"scale(0.015625)\"/>\n       </defs>\n       <use xlink:href=\"#DejaVuSans-38\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"xtick_6\">\n     <g id=\"line2d_6\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"251.365234\" xlink:href=\"#m982745fefd\" y=\"119.002734\"/>\n      </g>\n     </g>\n     <g id=\"text_6\">\n      <!-- 1000 -->\n      <g transform=\"translate(238.640234 133.601172)scale(0.1 -0.1)\">\n       <defs>\n        <path d=\"M 794 531 \nL 1825 531 \nL 1825 4091 \nL 703 3866 \nL 703 4441 \nL 1819 4666 \nL 2450 4666 \nL 2450 531 \nL 3481 531 \nL 3481 0 \nL 794 0 \nL 794 531 \nz\n\" id=\"DejaVuSans-31\" transform=\"scale(0.015625)\"/>\n       </defs>\n       <use xlink:href=\"#DejaVuSans-31\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"190.869141\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"xtick_7\">\n     <g id=\"line2d_7\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"294.958984\" xlink:href=\"#m982745fefd\" y=\"119.002734\"/>\n      </g>\n     </g>\n     <g id=\"text_7\">\n      <!-- 1200 -->\n      <g transform=\"translate(282.233984 133.601172)scale(0.1 -0.1)\">\n       <use xlink:href=\"#DejaVuSans-31\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-32\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"190.869141\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"xtick_8\">\n     <g id=\"line2d_8\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"338.552734\" xlink:href=\"#m982745fefd\" y=\"119.002734\"/>\n      </g>\n     </g>\n     <g id=\"text_8\">\n      <!-- 1400 -->\n      <g transform=\"translate(325.827734 133.601172)scale(0.1 -0.1)\">\n       <use xlink:href=\"#DejaVuSans-31\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-34\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"190.869141\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n   </g>\n   <g id=\"matplotlib.axis_2\">\n    <g id=\"ytick_1\">\n     <g id=\"line2d_9\">\n      <defs>\n       <path d=\"M 0 0 \nL -3.5 0 \n\" id=\"mbfaa1a1b94\" style=\"stroke:#000000;stroke-width:0.8;\"/>\n      </defs>\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"33.2875\" xlink:href=\"#mbfaa1a1b94\" y=\"10.999219\"/>\n      </g>\n     </g>\n     <g id=\"text_9\">\n      <!-- 0 -->\n      <g transform=\"translate(19.925 14.798437)scale(0.1 -0.1)\">\n       <use xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"ytick_2\">\n     <g id=\"line2d_10\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"33.2875\" xlink:href=\"#mbfaa1a1b94\" y=\"32.796094\"/>\n      </g>\n     </g>\n     <g id=\"text_10\">\n      <!-- 100 -->\n      <g transform=\"translate(7.2 36.595312)scale(0.1 -0.1)\">\n       <use xlink:href=\"#DejaVuSans-31\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"ytick_3\">\n     <g id=\"line2d_11\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"33.2875\" xlink:href=\"#mbfaa1a1b94\" y=\"54.592969\"/>\n      </g>\n     </g>\n     <g id=\"text_11\">\n      <!-- 200 -->\n      <g transform=\"translate(7.2 58.392187)scale(0.1 -0.1)\">\n       <use xlink:href=\"#DejaVuSans-32\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"ytick_4\">\n     <g id=\"line2d_12\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"33.2875\" xlink:href=\"#mbfaa1a1b94\" y=\"76.389844\"/>\n      </g>\n     </g>\n     <g id=\"text_12\">\n      <!-- 300 -->\n      <g transform=\"translate(7.2 80.189062)scale(0.1 -0.1)\">\n       <defs>\n        <path d=\"M 2597 2516 \nQ 3050 2419 3304 2112 \nQ 3559 1806 3559 1356 \nQ 3559 666 3084 287 \nQ 2609 -91 1734 -91 \nQ 1441 -91 1130 -33 \nQ 819 25 488 141 \nL 488 750 \nQ 750 597 1062 519 \nQ 1375 441 1716 441 \nQ 2309 441 2620 675 \nQ 2931 909 2931 1356 \nQ 2931 1769 2642 2001 \nQ 2353 2234 1838 2234 \nL 1294 2234 \nL 1294 2753 \nL 1863 2753 \nQ 2328 2753 2575 2939 \nQ 2822 3125 2822 3475 \nQ 2822 3834 2567 4026 \nQ 2313 4219 1838 4219 \nQ 1578 4219 1281 4162 \nQ 984 4106 628 3988 \nL 628 4550 \nQ 988 4650 1302 4700 \nQ 1616 4750 1894 4750 \nQ 2613 4750 3031 4423 \nQ 3450 4097 3450 3541 \nQ 3450 3153 3228 2886 \nQ 3006 2619 2597 2516 \nz\n\" id=\"DejaVuSans-33\" transform=\"scale(0.015625)\"/>\n       </defs>\n       <use xlink:href=\"#DejaVuSans-33\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n    <g id=\"ytick_5\">\n     <g id=\"line2d_13\">\n      <g>\n       <use style=\"stroke:#000000;stroke-width:0.8;\" x=\"33.2875\" xlink:href=\"#mbfaa1a1b94\" y=\"98.186719\"/>\n      </g>\n     </g>\n     <g id=\"text_13\">\n      <!-- 400 -->\n      <g transform=\"translate(7.2 101.985937)scale(0.1 -0.1)\">\n       <use xlink:href=\"#DejaVuSans-34\"/>\n       <use x=\"63.623047\" xlink:href=\"#DejaVuSans-30\"/>\n       <use x=\"127.246094\" xlink:href=\"#DejaVuSans-30\"/>\n      </g>\n     </g>\n    </g>\n   </g>\n   <g id=\"patch_3\">\n    <path d=\"M 33.2875 119.002734 \nL 33.2875 10.890234 \n\" style=\"fill:none;stroke:#000000;stroke-linecap:square;stroke-linejoin:miter;stroke-width:0.8;\"/>\n   </g>\n   <g id=\"patch_4\">\n    <path d=\"M 368.0875 119.002734 \nL 368.0875 10.890234 \n\" style=\"fill:none;stroke:#000000;stroke-linecap:square;stroke-linejoin:miter;stroke-width:0.8;\"/>\n   </g>\n   <g id=\"patch_5\">\n    <path d=\"M 33.2875 119.002734 \nL 368.0875 119.002734 \n\" style=\"fill:none;stroke:#000000;stroke-linecap:square;stroke-linejoin:miter;stroke-width:0.8;\"/>\n   </g>\n   <g id=\"patch_6\">\n    <path d=\"M 33.2875 10.890234 \nL 368.0875 10.890234 \n\" style=\"fill:none;stroke:#000000;stroke-linecap:square;stroke-linejoin:miter;stroke-width:0.8;\"/>\n   </g>\n  </g>\n </g>\n <defs>\n  <clipPath id=\"pc3fd492f59\">\n   <rect height=\"108.1125\" width=\"334.8\" x=\"33.2875\" y=\"10.890234\"/>\n  </clipPath>\n </defs>\n</svg>\n",
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAXcAAACPCAYAAAABdps+AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjQuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/Z1A+gAAAACXBIWXMAAAsTAAALEwEAmpwYAAAfvklEQVR4nO3dfZRcd33f8ff33nna50dJK+2uLdkWBoILfuDBIaTGhMcCpi0FU5/UUHpcEjiHBHLAQNs0p+k5gbYh0BCITyA1YDCUh+BSUkoMISHFsi0/yLJkSSt5d7WrXe3D7NM8z9z77R/37mokraRdaWfnYb+vc/bszL13d79zd+Yzv/ub370/UVWMMcY0FqfaBRhjjNl4Fu7GGNOALNyNMaYBWbgbY0wDsnA3xpgGZOFujDENqCLhLiJvEpEjIjIkIvdW4m8YY4y5MNnoce4i4gJHgdcDY8BjwHtU9dCG/iFjjDEXVImW+yuAIVU9oaoF4EHgjgr8HWOMMRdQiXDvB06W3R8LlxljjNkkkWr9YRG5B7gnvHtzterYLBKNE+0eAJFql2KMAfxCjlJyrNplXKkZVd222opKhPs4MFh2fyBcdhZVvQ+4D0BEGv4CN9Ftu+m769OIW7X3U2NMmfz4c0w+8HHwvWqXciVGLrSiEt0yjwF7RWSPiMSAO4GHKvB36ooW86hfqnYZxpgtYsObkapaEpEPAT8GXOArqvrsRv+deuOlkmg+C9FEtUsxxmwBFekjUNUfAT+qxO82xhhzaXaGqjHGNCALd2OMaUAW7sYY04As3I0xpgFZuG8S9T38Ur7aZRhjtggL902ihSze4ky1yzDGbBEW7sYY04As3I0xpgFZuBtjTAOycDfGmAZk4W6MMQ3Iwn3TKFrMVrsIY8wWYeG+WVQpztb9xADGmDpxyXAXka+IyJSIHCxb1i0iPxGRY+H3rnC5iMjnRWRIRA6IyE2VLN4YY8zq1tJy/x/Am85Zdi/wsKruBR4O7wO8Gdgbft0DfHFjyjTGGLMelwx3Vf07IHnO4juA+8Pb9wPvKFv+VQ08AnSKyM4NqtUYY8waXe5kHTtUdSK8PQnsCG/3AyfLthsLl01gjDFVoBpO0eyX0GIeLzWHFrJkT+wHbdzpm694JiZV1cuZ4FpE7iHoujHGmPPoeoNXFXwPP5/Gz6cpzZ+mND+Bl1mgOD2CpufoSc9xg5fjJY7whYU5C/dVnBaRnao6EXa7TIXLx4HBsu0GwmXnUdX7gPsALufNoR6p76GqiEi1SzGm5mkhy/wvHsDPZ9a0vZ9ZxFuaxksv4BcyaDFHRGHAdXhNPMGvxePcHIvT6zST9H2+LkLOwv08DwF3A38Ufv9B2fIPiciDwCuBhbLumy2vOD1c7RKMqRvFuQlST/yIqFekCKw1hhMI10Qi3N7SymvicV4cjdEhclajqt1x6HIcTvt+RWqvBZcMdxH5JnAb0CsiY8DvE4T6t0Xk/cAI8K5w8x8BbwGGgAzwvgrUXLfUK1W7BGPqhpdO0oXyhZ5eFn2fBV95rlRkzvcYLnkkfY9FVdK+T4vj8MJIlNcmErwmFmdPJELzOYF+LpfGPoK+ZLir6nsusOp1q2yrwAevtChjTGPRQpa++VNMbduDyqVHYKsq3sIUN0ci3BqL45aFtK+KB2RUyaoy7pXodVx2uS4RWFO3Zwy4NhLh2VLx8h9UjbMzVI0xFaWq9JzYz2f+5r9z0+TRNX+I6U2P8M7m5vNCyhEhKkKH49Dnutwci3N1JEL0Ei31c0Ua/LMvC3djTEWJV+Ltz+/jmmKWj/3y6+yZP7WmgN9WzPLyWLxiAxAGXLciv7dWWLgbYyqqd/IYb5s+joiwKzXDRx59kN7swsUD3itxS2qaHqcyESUibLdwNxumgT+ZN2ZVvs/bnt/HdvUAEOBlp4/x4Ue/RUc+jeN7QciXBb2q4hbSvIMSTgW7ThIN3i1zxScxmbUrzo2jpQISjVe7FGM2xY7ULG8bf+asrhUBfn30KV40O8LJ9u3k3BjPd+1iMdrEkWKRTC5FbuIYNxTSFa3t2kiECNCoY9gs3DdRMBSycU+aMOYsvsdbj/8/thXOPwnJRelLJ+lLB5etes3YAZRwlAxQUKVZBCrYum70bgsLd2PMxlBFgPZ8mt2zw9x87Jf8s/Fn1jyaXAj6wh0gugldJjscl1ZxmNfG7C61cDdmI6giKCBog/flnkWViO/Rk13ghunj3DLxHDdPHKE3O0/E8yrZ8L5sChQlykLbtfQVu5ifO1ztkirCwt2YDeCqz4cf+zZNxTwzzR0U3CgnOvvxRZhs7WGmqYO5RFtFuxk2jSqJUoGrFyZ5xcQhbp54juvmxmnPp3HKux1r7KEqkHJbOd68lyfbb2Y6vp1E7vtg4W6MuRBfhM5cittH9gNnf7KSd6NMtXTzyds+wHBHX90FvKhPvFSkK7fE3rmT3Dh5lJsmj9K/NEXcK9Zahp9FVfHFZTy6jUPtL2Wo9UUsue0ggqpS9PLVLrFiLNw30zlDvkzjUHGYT7SGHTNnN1oTXpHBxdN8dN83+IPXvJ+Zpo7aDPjwuRn1S3Tk0+xamuYl0yd4QfIkuxcm2J6eo7WQPbt1XqMUyBBhqHk3z7S9lMmW6yhI7KxRO4oyl52sXpEVZuG+ibzMPF5mASfWVO1STAWc6Nx1wXXL47s/+vP7+Pe3vIti79XVv/Rz2F/enk+zKzXDdXNjXD87yjXz4/SlkrQX0kR8r6Zb5ufyFKbdFg42XcfxzluYje1AxAWRunocG8HCfRPFfR/xSkELqdovbLPhspGLn78gwK9OHeeeh/+MP3vtv0W3X1P5gC87Uox5RRJeke3pJNfMn+L62VFekBzlqoVJ2guZugvyZapKXhxGI7080vIrTLbfQCnaiWzBQC+3lkv+DgJfJZhKT4H7VPVzItINfAvYDQwD71LVOQmerZ8juPRvBnivqj5RmfLrhwP859YWep/4Hsf69nKsayD4oK25k0wkgV+h06zPokp3bpH5eCu+09inXlfDaMcOSo5L1PcuuI0rwr/ILzD0kz/lf916F7Frbr7igHd8H0d9mkp54qUC3bkl2gppdqZm6cyl6F+aoiOfZkc6SUc+TVduiZgXXA2xnsNPFaZwOZzYzaH2l7LYshffiW35UF+2lpZ7Cfioqj4hIm3AfhH5CfBe4GFV/SMRuRe4F/g48GZgb/j1SuCL4fctzQfyvsftE4d47cQhFCEXiTHb1MFo+3aGugc52j3IyfbtzDR3kY4m8MTZ0BZ+d3aRP/z5ffzlda/m0eturX63QIPJRBKXDHcIxnD/XinN7D98ncf6X4gkWi/+i8v6wtvyGbZl57l6YZId6SQ70kl6Mgv0Zhdoz6dpLWSJewWiXgkHbbiQUyDvxJmI7+RgbJDDzXspJvpwnCDK1vN4fb9EoZStSJ21YC3Xc58gnOBaVZdE5DDBpNd3EEziAXA/8LcE4X4H8NXw2u6PiEjn8pR8G19+fUmrrkyzJyjNpTzNS1MMLk3xq+MHUYSiG2Eu0cbJtu0MdQ9wpPsqhjt3kmxqZzHWQmm5xb3OYI6VCnzgyb/ihunjfGBxisN9e0m1bbvsxxL1ipTEQe0IYMVscweLsRaaSoVLbtvsOPy7qPCf5sZ4ou/6M//PcLx8olSgI5+mf2ma65InuT55kp2pGXalZmjLZ4j6wUnzjRbe51JVighzThOT8Z0cb76W6Za9zEe78AkaP5d7zFv082QKCxtaby1ZV5+7iOwGbgT2ATvKAnuSoNsGguA/WfZjY+Gys8J9K06QfbR44YkBghEWStwrrpyWfcvkcwDk3RipWBOnWns53tXPkZ6rON7Vz2RLD+loEwU3ctGwd9Tnnx/5W954Yh8C7M0v8c5jv+D+G+9Y08QJ52rLp/ndfd/kp9v38ovrf90+Pwh54pCPxNa8fW8xy72//BqfvO0DCDCwOMULkicZWJxi98IEvZl5mkp5XPUbPsSXqSo+sKjwvNPCiVgf020vYrHlGvxoB8r6rtm+la053EWkFfgu8DuqunjWkCJVXe8k11txguz0OodBLu/hhFcgkS3Qm13gH00fD86wcyIsxluYau7iRNcujnUPMtQ1wHhbL4vxVgrOmX/tK8ef5X1P/2/c8DRrB3j30b/nscGX8mzvnrWHsyqthSwfe+QBbht5gpedPkayZ5BD6/kdDSwXiTHR2sNVi6fXtL0AO1OzfP7/fpa4VyTqlZAG7Eq5GFVFCV4bYz48RTMnE4Nk238Fad2DG2lBygJ9K+2bK7WmcBeRKEGwP6Cq3wsXn17ubhGRncBUuHwcGCz78YFw2ZY3XCqRBxJX+HsEiPklerNBX+uLZ4dRoOS4pGLNzDR1MNzRx9Geqxhr284H93+XptLZJ2u0FjL89v7v8bHbP0h6jUMzW4tBsP/jkScQYFt2gY88+iCfuO23mG7pusJHVf8UYbapfV0/I0D7KhfWamRBVwuMex6HPThAC6dariPW+RJamweIRZqIXcYRpTnbWkbLCPBl4LCq/nHZqoeAuwkmy74b+EHZ8g+JyIMEH6QuWH97IIfiV2gYpABR36MrtxSeSTjGbww/ji+Co+e3BgW4Yeo4bx36B771otddvCZVWotZfu+Rb3DbyP6VPk4Brp8d5YP7v8tnXnUXmUYYv3/O0VVEPdzwA1JHla7cEk54BNSdXaQrtxRs53tcOzfO4OIU5myqSgmY9ZXjvsNBmjke6SbZOkB7+/W0N+2gz01sendLrpim6F/685F6tZaW+6uB3wSeEZGnwmWfJAj1b4vI+4ER4F3huh8RDIMcIhgK+b6NLLiezfs+8+pz/qyQlSGAe5GuIAflPc/+hH29exhu6cGJtyCrDMlsLub46L5vcvvw4+dVLsDtw/uZberg51e9bCPLByATTTDb1LGmbQtulNwF+rxjXpFE+EGnoz596eRKaPdkF9mWmQMg4vtcOzdGJJxcoiu7RHd2Mfg5fHqyi0jZ6JXIOSNjrNsgCPMCwow6TEXaGXK7GI7tYC6+ExLbScQ66HTidLK2yawrJV/KUGrgcBetgdPht0qfe5MI/2fbdq6JRKtdygoFRp0YM26ESFsvskptLYUse+ZPXfQtyQf8ChxKl5wImTVObpKKNTN/gWGFbfkMHfkUABIeiSyHtKNaF6fU16LlvVaUCAtOC5PRLo5HtzMa28F8bDsS68Zx42f1m9cCVWVicYgfPPPf0Pq+5O9+Vb1ltRV2huomKqgy5nk1Fe4CXO0XuNovQHL0sn+PAyvdFRsp4hVIeGtrXXXnltb8Yaa5PKrBqKCM20Iy1sPpWB+T8Z1Mx7Yx57ZTcuMoDiJC7TzLz/B9j4XcNM/PPsWxqX31HuwXZeG+iTxgyeZRNTVueQRLVpWUKtM+nHKayMR34LXsYbH5ahZj28i6zcHYnnNa5LXTPg+oKrlSmpNzz3J8Zj/j80coeI178tIyC/dNdrzkrVw50JhqWh61Mu/7LPo+457H8VKJER+SbivpRD/xlkGaWnfT0tRHItqKhK3yWqeqlPw8U0vDHJt+jLH5wyzlZtEt1P1m4b7JHml5Edf33sBV2RF25Cdo85aIaMnC3lTUcms8pcpwqcQTxQJPFwocK5WYUoFoO83NA/T17GVH+7V0N+1gZ6QJaqyv/GKCx+gznznNSPIZjk4/ynxmAk8bdQrsi7Nw32RLbjtPtN/CE+23EPfzdBWT9OfHGMyO0pefoM1bxNX6vDqfqR3LAyWyqox7HgeKBX6ay/FMscgUDtFYN9tbr2dnx16ubemns2kHsUhzzX3weSnLjzNTWODk/CFOzDzJqYWjW6Lb5VIs3KshfPHk3QST7i4m4zvZ3/5y4n6O7mKS/twYg7lRduQnLezNmi13s0x5HodKJX6Wz/F00eOUuviRFnrb9jLYsZdb2vbQkdhG1A1Op6unMF8WzKKUYzo1ypGpRxibP0wqn6x2WTXFwr0WrIR9ExNuPxOJfvbry4n7ebqLs+zKjXFVGPatpSVcLOy3kvJeYg8XFaEkUbJOAhWHuWg3BRye0yjH3E5OeB5uog9xIrzATXBjrJOomyDmxpE6PvNTVfG0xHxmkpG5gwxNP8ZcZhJ/i3a7XIqFe41Scci5TZxyBzgV72d/xyuI+zl6irNByz47yvbCJK2llIV9nVPAxyHvJMg7cdJuCwgkoz0UnDgZp4m5aDcA89EuCk6MokTD0SpQkijBtSQDu+uwJX4hy4G+kD3NSPIgw8kDzKZPNvTcpxvFwr0eSHA5qZzbzLjbzHhikMc6XknCz9FdmKWvMEF7aYGO4gIxLdBZDM62bPFSOOqj6hFVPzhACPso6/FQvBGcOeknxmKkjanYDk4l+pmM72Iu2k1JIhSc4Czb5UvarlWj/EdVFV9LzGenGJ17luHZp5ixQF83C/c6peKQdZsZb2pmvGnwTGijRMLD1GYvjasexVKKHX4ar5RjKTvJCyMRlrITeH6RbV6KZi3gqUenFmgTB/BpD2ezcQhmDzKXZ7lVnnWD1vdkfBfjiQGmYttZjHRQlPBUny2+j4NA91jITjE6d5DnZ58OAz1X7dLqloV7owjDQRGKErT8FsIWILFelpa364S/Vw3H+yqel0fVo+gVoLhIVCCdm6FX85S8LE5uin7XJZVPchUFEnhoMcWeSATfL9ImSqs4gNJ0zsQJW+noYPk65FmFJC75aDuL8T6m47uYaRpgLtpNxm1Zd2u8kQVj0Qsk06eYWDzGcPIA06lRC/QNYuG+BQUzQQUB40SaAYhFgURPcLt1Dx5AGFgnAV89ZtTH0xKFYpqYQCo/T8zPEfVzLGQn2e1GyOZniHsZrnHPf2o1Ow7XRSIV6T5oLy0Q9S88GcrlKgGjpRLnnlecVeVYqbjSzZKOtJNuGiTT1I/TPEA01glO/Y5GqZSVQM+cYiT5DCPJAyQzwVGk2VgW7uaCykPJleCpEiFGPHxDiCfOTNOXALKAqk8WeHa13wdUaqb0qBYqcm0bBVL+auc16lmBH3HjuBIhYUF+HlXF84skMxOMJA8wknyGZGackgV6Ra3leu4J4O+AeLj9d1T190VkD/Ag0APsB35TVQsiEge+CtwMzALvVtXhCtVvaoyIc9GWeaVO/i7IlU6BcmFRmyZ23ZYDfS47wUjyGYZnD5DMnGroS+zWmrW03PPA7aqaCmdk+oWI/DXwEeCzqvqgiHwJeD/wxfD7nKpeJyJ3Ap8G3l2h+o0xNSA4U1QplLLM56Y4OXeIkeQBZtPjFuhVcslw1+C/lgrvRsMvBW4H/mW4/H7gPxKE+x3hbYDvAH8qIqK1cOF4Y8yGWAlzL8d89jSnF59nYvEY06lR0vm5LXs9l1qy1jlUXYKul+uALwDHgXnVlf/gGNAf3u4n+AwOVS2JyAJB183MOb/zHuCeK30AxpjNoepT8HIsZKc4vfQ8pxaOWpjXsDWFu6p6wMtEpBP4PvDCK/3DqnofcB9snZmYjKkXywfa5WE+sXiM6aURUoU5G91SB9Y1WkZV50XkZ8CtQKeIRMLW+wAwHm42DgwCYyISAToIPlg1xtSwYJhikVR+llMLRzm1cIzp1AhL+aSFeR1ay2iZbUAxDPYm4PUEH5L+DHgnwYiZu4EfhD/yUHj/l+H6n1p/uzG1Z/n65+nCArOpMcbmD3Nq4SiLuWkKdiJR3VtLy30ncH/Y7+4A31bVH4rIIeBBEflD4Engy+H2Xwa+JiJDQBK4swJ1162lfJLx+SO0J3pxxEXEWRk3DoTX066P2W5M/VnuN5/PTDI2/xwTi0NMpUbIF1NbapairUBqoVG91frcBYdYJIEgOOLSGu9eOSU94sToatpx1inqTdE2OsIThhwnQldTHyIOLbEOHHFxnRhOeClXe1Mw5ZavqriUm2FqaYTRuYNMLQ1bv3nj2K+qt6y2wsK9DrkSARGaoq0ILm2JbqJunLZ4D82xdhLhm0FbooeWWAcRJw5Y8G8Fy10tuWKK6dQoY/PPMbl4nGTmlF2zpTFdMNzt8gN1yNMSKKTywaV9l/Iz520jCFE3QWu8i22tV9HbejW9LQN0Ne8kHmkOu4Qs7OvJ8tjyciW/iK8eqj4LuWlOL55gdO5ZZtNjZIqLaAUuyWDqg7XctxBHXBLRVjoS2+hpGWBnx166m3fSEusiXmeTITeCM689xQsDOhf2fafz8+S9DJnCIql8MNjMV59kehxPvZXfkSkskC9lUFVyxSUbb771WLeMWV3EidEca6enZZBtrYMkIq1AcI2YruaduM75B3cxN0FTtG3lvutEiDgx7M3hDA0vq6zqU/RyqPqk8nP46jGfPU3Ry5MtpljMTVP08yxkp1D1SRfmUfVXWuTGXIKFu1m/4CJg54e160SJuWcu1BWPtNCW6Ka7uZ+eln6aY+20x3tpirXhOrFwBFB4vfnznm+K55fK7im5Yuqygy0RbV25gmVQa4Rz5yi63Deg8to9v4iiZItL+OqTys9SKOVIF+ZI5efJl9IsZKfxtMhSbhZFyRfTKGqhbTaShbvZPCIOUSdOc6yDjsQ22pu2rYz2WcrPki4srGxb8gos5KbO/LAqmeIivn8ZASjQFG1fOdoQhI6m7bhOMNtR1InR3dK/6hvWWhS8HHOZiXCS5tOoeiu1lrRo/dumGizcjTGmAV0w3J3VFhpjjKlvFu7GGNOALNyNMaYBWbgbY0wDsnA3xpgGtOZwFxFXRJ4UkR+G9/eIyD4RGRKRb4lILFweD+8Phet3V6h2Y4wxF7CelvuHgcNl9z9NMEH2dcAcwcTYUDZBNvDZcDtjjDGbaE3hLiIDwD8B/iK8LwQTZH8n3OR+4B3h7TvC+4TrXyd2Troxxmyqtbbc/wT4GLB8Cl4Pa5wgG1ieINsYY8wmuWS4i8hbgSlV3b+Rf1hE7hGRx0Xk8Y38vcYYY9Z2PfdXA28XkbcACaAd+BxXOEG2qt4H3Ad2+QFjjNlol2y5q+onVHVAVXcTzIf6U1W9izMTZMPqE2SDTZBtjDFVcSXj3D8OfCScCLuHsyfI7gmXfwS498pKNMYYs152VUhjjKlfdlVIY4zZSizcjTGmAVm4G2NMA7JwN8aYBmThbowxDcjC3RhjGpCFuzHGNCALd2OMaUAW7sYY04As3I0xpgFZuBtjTAOycDfGmAZk4W6MMQ3Iwt0YYxqQhbsxxjQgC3djjGlAa5lDdTOkgCPVLmIdeoGZahexRvVUK1i9lVRPtUJ91VutWq++0IpaCfcjF5pNpBaJyOP1Um891QpWbyXVU61QX/XWYq3WLWOMMQ3Iwt0YYxpQrYT7fdUuYJ3qqd56qhWs3kqqp1qhvuqtuVpFVatdgzHGmA1WKy13Y4wxG6jq4S4ibxKRIyIyJCL31kA9gyLyMxE5JCLPisiHw+XdIvITETkWfu8Kl4uIfD6s/4CI3FSFml0ReVJEfhje3yMi+8KaviUisXB5PLw/FK7fXYVaO0XkOyLynIgcFpFba3zf/m74PDgoIt8UkUQt7V8R+YqITInIwbJl696fInJ3uP0xEbl7E2v9L+Fz4YCIfF9EOsvWfSKs9YiIvLFs+aZkxmr1lq37qIioiPSG96u6b1elqlX7AlzgOHANEAOeBl5c5Zp2AjeFt9uAo8CLgc8A94bL7wU+Hd5+C/DXgACvAvZVoeaPAN8Afhje/zZwZ3j7S8Bvhbd/G/hSePtO4FtVqPV+4N+Et2NAZ63uW6AfeB5oKtuv762l/Qv8OnATcLBs2br2J9ANnAi/d4W3uzap1jcAkfD2p8tqfXGYB3FgT5gT7mZmxmr1hssHgR8DI0BvLezbVevfjD9ykZ13K/DjsvufAD5RzZpWqfEHwOsJTrLaGS7bSTA2H+DPgfeUbb+y3SbVNwA8DNwO/DB8cs2UvWBW9nH4hLw1vB0Jt5NNrLUjDEs5Z3mt7tt+4GT4woyE+/eNtbZ/gd3nBOa69ifwHuDPy5aftV0laz1n3T8FHghvn5UFy/t2szNjtXqB7wAvBYY5E+5V37fnflW7W2b5xbNsLFxWE8LD6huBfcAOVZ0IV00CO8Lb1X4MfwJ8DPDD+z3AvKqWVqlnpdZw/UK4/WbZA0wDfxl2I/2FiLRQo/tWVceB/wqMAhME+2s/tbt/l613f1b7ObzsXxO0fqFGaxWRO4BxVX36nFU1V2+1w71miUgr8F3gd1R1sXydBm/BVR9mJCJvBaZUdX+1a1mjCMFh7hdV9UYgTdBtsKJW9i1A2Fd9B8Gb0i6gBXhTVYtap1ranxcjIp8CSsAD1a7lQkSkGfgk8B+qXctaVDvcxwn6r5YNhMuqSkSiBMH+gKp+L1x8WkR2hut3AlPh8mo+hlcDbxeRYeBBgq6ZzwGdIrJ8aYnyelZqDdd3ALObVCsErZYxVd0X3v8OQdjX4r4F+A3geVWdVtUi8D2CfV6r+3fZevdnVfeziLwXeCtwV/hmxEVqqmat1xK80T8dvuYGgCdEpO8idVWt3mqH+2PA3nD0QYzgQ6iHqlmQiAjwZeCwqv5x2aqHgOVPuu8m6ItfXv6vwk/LXwUslB0SV5SqfkJVB1R1N8G++6mq3gX8DHjnBWpdfgzvDLfftFadqk4CJ0Xk+nDR64BD1OC+DY0CrxKR5vB5sVxvTe7fMuvdnz8G3iAiXeHRyhvCZRUnIm8i6FZ8u6pmznkMd4YjkPYAe4FHqWJmqOozqrpdVXeHr7kxgsEXk9Tgvq14p/4aPrB4C8GIlOPAp2qgnl8jOIw9ADwVfr2FoO/0YeAY8DdAd7i9AF8I638GuKVKdd/GmdEy1xC8EIaA/wnEw+WJ8P5QuP6aKtT5MuDxcP/+FcEIgprdt8AfAM8BB4GvEYzeqJn9C3yT4POAIkHYvP9y9idBf/dQ+PW+Tax1iKBPevm19qWy7T8V1noEeHPZ8k3JjNXqPWf9MGc+UK3qvl3ty85QNcaYBlTtbhljjDEVYOFujDENyMLdGGMakIW7McY0IAt3Y4xpQBbuxhjTgCzcjTGmAVm4G2NMA/r/cILuq3nEcOsAAAAASUVORK5CYII=\n"
     },
     "metadata": {
      "needs_background": "light"
     }
    }
   ],
   "source": [
    "import numpy as np\n",
    "from matplotlib import pyplot\n",
    "from PIL import Image\n",
    "\n",
    "def create_area_mask(image, segs):\n",
    "    mk = np.array([])    \n",
    "    #verify image & segs have the same integer dimensions\n",
    "    if image.shape == segs.shape and segs.dtype == image.dtype and np.issubdtype(np.uint8,segs.dtype) :\n",
    "        mk = segs\n",
    "    \n",
    "    return mk\n",
    "\n",
    "# load image as pixel array\n",
    "image = np.array(Image.open('Aconitums004.tif'))\n",
    "\n",
    "# Verify image is correct: summarize shape of the pixel array\n",
    "print(image.dtype)\n",
    "print(image.shape)\n",
    "\n",
    "# Verify image is correct: display the array of pixels as an image\n",
    "\n",
    "pyplot.imshow(image)\n",
    "pyplot.show()\n",
    "\n",
    "# load image as pixel array\n",
    "# Be aware that matplotlib uses by default png.read_png_float for png images, \n",
    "# use PIL instead:\n",
    "segs = np.array(Image.open('Aconitums004.png'))\n",
    "\n",
    "\n",
    "\n",
    "mask = create_area_mask(image, segs)\n",
    "if mask.all():\n",
    "  print(\"Mask is empty\")\n",
    "else:\n",
    "    # Verify mask is correct: summarize shape of the pixel array\n",
    "    print(mask.dtype)\n",
    "    print(mask.shape)\n",
    "\n",
    "    # Verify mask is correct: display the array of pixels as an image\n",
    "\n",
    "    pyplot.imshow(mask)\n",
    "    pyplot.show()\n",
    "\n",
    "\n",
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.4"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}